In [5]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [6]:
from src.curve import build_zero_curve, discount_factor, shift_curve
from src.pricing import price_from_curve

In [7]:
# Courbe jouet : maturités (années) et taux continus correspondants
mats = np.array([1, 3, 5, 10], dtype=float)
y_cc = np.array([0.02, 0.022, 0.025, 0.027], dtype=float)

# Construction de r(t)
r = build_zero_curve(mats, y_cc)

# Vérif rapide : r(5y) doit être ~2.5%
r(5.0)

np.float64(0.05449999999999999)

In [ ]:
import pandas as pd
from pathlib import Path

# Dossier data (à la racine du repo)
data_path = Path("..") / "data"
data_path.mkdir(parents=True, exist_ok=True)

# Exemple minimal : 3 obligations plain vanilla (coupons annuels)
pf = pd.DataFrame({
    "id":      ["BOND_ZC", "BOND_A", "BOND_B"],
    "face":    [100,       100,      100],
    "coupon":  [0.00,      0.03,     0.05],
    "freq":    [1,         1,        1],          # 1 = annuel
    "maturity":["2028-12-30","2030-12-30","2035-12-30"]
})

csv_file = data_path / "portfolio.csv"
pf.to_csv(csv_file, index=False)
csv_file


PosixPath('../data/portfolio.csv')

In [9]:
import pandas as pd
pf = pd.read_csv("../data/portfolio.csv") 
pf


,id,face,coupon,freq,maturity
0,BOND_ZC,100,0.00,1,2028-12-30
1,BOND_A,100,0.03,1,2030-12-30
2,BOND_B,100,0.05,1,2035-12-30


In [10]:
from src.pricing import price_from_curve

T = 5.0
P_base = price_from_curve(r, [(T, 100.0)])

r_up100 = shift_curve(r, 100)  # +100 bps parallèle
P_up100 = price_from_curve(r_up100, [(T, 100.0)])

P_base, P_up100, P_up100 < P_base  


(76.14734291752053, 72.43359318069157, True)